# Regression

This notebook is an example of how to employ `ortho-svm` to perform regression tasks using **custom kernels** with Support Vector Machines. Althought `scikit-learn` already provides an API for this, `ortho-svm` makes it easier to do as it already has a couple of pre-defined kernels such as:

- **Hermite kernel:** A special kernel defined by Hermite polynomials.
- **Gegenbauer kernel:** A family of kernels defined by a parameter $\alpha$ that generalizes most of the classic polynomials, e.g. Legendre and Chebyshev.
- **Chebyshev kernel:** A particular instance of the Gegenbauer kernel when $\alpha = 0$; the Chebyshev polynomials defined are the Chebyshev polynomials of the _first kind_.

In this document, an example of a **regression** task will be performed by comparing the results obtained with those from the literature.

## 1. Setup

We will be performing regression for the Forest fires dataset [2] using the Gegenbauer kernel defined in `ortho-svm`.

The **forest fires** dataset has the following properties:

- 517 instances
- 12 attributes (categorical and numerical)
- 1 prediction attribute

The regression problem is a hard one, and we will be comparing closely with the reference paper [1]. In this case, there is no simple way of selecting the hyperparameters, we we will be performing _random search_ for the following parameters:

- $C \in [0, 100]$
- $\varepsilon \in [0, 1.0]$
- $\alpha \in (-0.5, 0.5]$
- $n \in [2, 10]$

We will be creating probability distributions to sample these values and obtain a good estimate, after all we are performing a proof-of-concept for this kernels.

## 2. Methodology

Here, we will be performing the following steps, given that we do not have a _test_ dataset:

1. Pre-process the data, including loading, cleaning and creating a splitter for the following steps. Split the dataset 80%-20%, and leave the 20% as a _test_ dataset.
2. Define a 10-fold cross validation the obtain the best hyper-parameters using the 80% portion of the original dataset.
3. Record all the values for the **mean-squared error** and keep track of the best parameters.
4. Finally, using the _test_ dataset we predict the needed values and then plot the final results.

## 2.1 Pre-processing

In [1]:
import numpy as np
from scipy.stats import uniform, loguniform
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from orthosvm.gramian import gram

First, we load the _training_ dataset, which is contained in the same directory as this document.

In [12]:
# We ignore the last column here because we don't need them, so we
# define a list of columns to use
dtypes = ["i4", "i4", "S3", "S3"]
dtypes += 9 * ["f4"]
raw_data = np.genfromtxt("forestfires.csv", delimiter=",", dtype=None, encoding=None)

# From this, we extract the labels
y = raw_data[:, -1]

# We now extract the features
xdata = raw_data[:, :-1]

# Check number of instances and features
print(xdata.shape)

[['X' 'Y' 'month' ... 'wind' 'rain' 'area']
 ['7' '5' 'mar' ... '6.7' '0' '0']
 ['7' '4' 'oct' ... '0.9' '0' '0']
 ...
 ['7' '4' 'aug' ... '6.7' '0' '11.16']
 ['1' '4' 'aug' ... '4' '0' '0']
 ['6' '3' 'nov' ... '4.5' '0' '0']]
(518, 12)


## 2.2 Perform a 5-fold cross validation

We now take the cleaned-up data and perform a grid search for the best possible value for $C$.

In [3]:
# First, we need to create our estimator, which is a special SVM for classification

# We define the function that will compute the Grammian matrix
# and we specify the hyper-parameters
gram_matrix = gram.gram_matrix(kernel="gegenbauer", degree=2, alpha=0.38)

# Next, we pass a dictionary of parameters, in this case the kernel, to the SVC
# estimator and we create an object with it
params = {"kernel": gram_matrix}
# params = {"kernel": "rbf"}
svc = SVC(**params)

In [4]:
# Now we proceed to create the GridSearchCV

# Create the grid to search
c_grid = [{"C": [i for i in range(10, 101)]}]
# Instantiate the grid search object
gcv = GridSearchCV(svc, c_grid, "accuracy", n_jobs=-1)

### 2.3 Train the model

We now proceed to train the model using the set-up we have until now, and with the great `scikit-learn` API, after running `GridSearchCV` we obtain the best estimator for the data.

**WARNING:** This might take a while, so if you have the processing resources to spare, enable the `n_jobs` parameter to make this easier for you.

In [5]:
# We proceed to fit the model to our data
# we use the n_jobs parameter before to make up for the time spent here
gcv.fit(xdata, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale',
                           kernel=<function gram_matrix.<locals>.compute_gram_matrix at 0x7f19364640e0>,
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'C': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
                                22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                                34, 35, 36, 37, 38, 39, ...]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [6]:
# Now let us compare the best value found for C
gcv.best_params_

{'C': 29}

So we are somewhat far from the estimated value for $C$ given in the main reference, but this will suffice for now. We can then use the best estimator found and try out the _test_ dataset with it.

### 2.4 Test the model

We now proceed to test the model, we first load the testing dataset and then we proceed to find the classification accuracy with the trained model.

In [7]:
# We do the same as before to load the information, we can even re-use the same variables
raw_data = np.loadtxt("monks-1.test", usecols=[i for i in range(7)])

# Again, we extract the labels
y = raw_data[:, 0]

# We now extract the features
xtest = raw_data[:, 1:]

# Let us now test our model
y_predict = gcv.predict(xtest)
# And print out a classification report
print(classification_report(y, y_predict))
print(accuracy_score(y, y_predict))

              precision    recall  f1-score   support

         0.0       0.84      0.70      0.77       216
         1.0       0.75      0.87      0.80       216

    accuracy                           0.78       432
   macro avg       0.79      0.78      0.78       432
weighted avg       0.79      0.78      0.78       432

0.7847222222222222


## References

1. Tian, M., & Wang, W. (2017). Some sets of orthogonal polynomial kernel functions. Applied Soft Computing, 61, 742-756.
    
2. Forest fires dataset: https://archive.ics.uci.edu/ml/datasets/Forest+Fires